In [13]:
import pandas as pd
import numpy as np
from pandas import Series,DataFrame
import numpy as np
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.multioutput import MultiOutputRegressor
from sklearn.neighbors import KNeighborsRegressor
# data file name  /dataAll.csv  /testAll.csv
data_train = pd.read_csv('dataAll.csv')
'''
# 去除未被覆盖的数据
k = np.array(data_train)
a = []
for i in range(len(k)):
    if data_train[i,:].max() <= -105:
        a.extend([i])
data_train.drop(a)
#data_train.head()'''
data_train_add = data_train.iloc[:,:30].applymap(lambda x: x)#+126.23
train_loc = data_train.iloc[:,30:]
data_train_add.to_csv("./processed_data/train_pro.csv",index = False)
m = data_train_add.describe()
m.to_csv("./processed_data/des_pro.csv",index = False)
train_loc_list=[]
for i in range(len(train_loc)):
    train_loc_list.append([train_loc.iloc[i,0],train_loc.iloc[i,1]])
#len(train_loc_list)

def cos_vali(data_list,class_list,test_size = 0.2):
    data_class_list = list(zip(data_list, class_list))
    import random
    random.shuffle(data_class_list)
    index = int(len(data_class_list)*test_size)#+1
    train_list = data_class_list[index:]
    test_list = data_class_list[:index]
    train_data_list, train_class_list = zip(*train_list)
    test_data_list, test_class_list = zip(*test_list)
    return list(train_data_list),list(test_data_list),list(train_class_list),list(test_class_list)
# Create a random dataset
def error(y_test,y_pre):
    m = y_test-y_pre
    m=np.square(m)
    m=m.sum(axis=1)
    m=np.sqrt(m)
    return m.sum()/len(y_test)

## 依据相关性来选取sector

In [14]:
a=data_train_add.columns.values.tolist()
m=[0,15,1,16,2,17,3,18,4,19,5,20,6,21,7,22,8,23,9,24,10,25,11,26,12,27,13,28,14,29]
order=[]
for i in m:
    order.extend([a[i]])
reoder_data = data_train_add[order]

def getReorderData(index):
    a = reoder_data.columns.values.tolist()
    order=[]
    for i in index:
        order.extend([a[i]])
    #return_data = 
    return reoder_data[order]

# 计算覆盖率
def getCoverRate(Sects):
    Sects = np.array(Sects)
    index = Sects*2
    index = np.hstack([index,index+1]).tolist()
    select_data = getReorderData(index)
    col_len = len(select_data.columns)
    row_len = len(select_data)
    select_data_1 = np.array(select_data)
    cover_index = []
    for i in range(row_len):
        if select_data_1[i,:].max() <= -105:
            cover_index.extend([i])
        
    drop_index = []
    '''
    for i in range(row_len):
        if select_data_1[i,:].max() <= -126.23:
            drop_index.extend([i])
    '''
    num = len(cover_index)
    rate = 1-num/row_len
    data = select_data.drop(drop_index)
    return rate,data

In [15]:
def train_pred_rf(select_data,train_loc_list):
    X_train, X_test, y_train, y_test = cos_vali(np.array(select_data).tolist(),train_loc_list)
    X_train, X_test, y_train, y_test = np.array(X_train),np.array(X_test),np.array(y_train),np.array(y_test)
    #regr_multirf = MultiOutputRegressor(RandomForestRegressor(n_estimators=400,
                           # max_depth=29,min_samples_leaf=2,oob_score=True))
    regr_multirf = MultiOutputRegressor(RandomForestRegressor(n_estimators=400))
    #regr_multirf = MultiOutputRegressor(KNeighborsRegressor())
    regr_multirf.fit(X_train, y_train)
    y_multirf = regr_multirf.predict(X_test)
    #y_multirf = regr_multirf.predict(test_1)
    #np.savetxt('BL_ans.txt', y_multirf, fmt='%f')
    a = pd.DataFrame(np.array(y_multirf))
    a.to_csv('baseline_rf_frat.csv',header=None)
    m=error(y_test,y_multirf)    
    return m
#train_pred_rf(select_data,train_loc_list)

def pred_rf(select_data,train_loc_list):
    X_train, X_test, y_train, y_test = cos_vali(np.array(select_data).tolist(),train_loc_list)
    X_train, X_test, y_train, y_test = np.array(X_train),np.array(X_test),np.array(y_train),np.array(y_test)
    #regr_multirf = MultiOutputRegressor(RandomForestRegressor(n_estimators=400,
                           # max_depth=29,min_samples_leaf=2,oob_score=True))
    regr_multirf = MultiOutputRegressor(RandomForestRegressor(n_estimators=400))
    #regr_multirf = MultiOutputRegressor(KNeighborsRegressor())
    regr_multirf.fit(X_train, y_train)
    #y_multirf = regr_multirf.predict(X_test)
    y_multirf = regr_multirf.predict(test_1)
    #np.savetxt('BL_ans.txt', y_multirf, fmt='%f')
    a = pd.DataFrame(np.array(y_multirf))
    a.to_csv('baseline_rf_frat.csv',header=None)
    m=error(y_test,y_multirf)    
    return m

In [17]:
#for index in [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14]:
def test():
    index = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14]
    print('###################################################')
    print(index)
    rate,select_data = getCoverRate(index)
    print(rate)
    print(train_pred_rf(select_data,train_loc_list))
    
test()

###################################################
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14]
0.9720424540512556
1.0269669627161433


In [8]:
m = [
    [0,1,4,5,6,7,8,9,10],
    [0,1,2,4,5,6,7,8,9,10],
    [0,1,4,5,6,7,8,9,10,14],
    [0,1,4,5,6,7,8,9,10,12],
    [0,1,4,5,6,7,8,9,10,11],
    [0,1,2,3,4,5,6,7,8,9,10],
    [0,1,4,5,6,7,8,9,10,11,12],
    [0,1,2,3,4,5,6,7,8,9,10,12],
    [0,1,2,3,4,5,6,7,8,9,10,12,13],
    [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14]
]

for index in m:
    print('###################################################')
    print(index)
    rate,select_data = getCoverRate(index)
    print(rate)
    print(train_pred_rf(select_data,train_loc_list))

##########################################
[0, 1, 4, 5, 6, 7, 8, 9, 10]
0.9593580119078436
1.7544326621392756
##########################################
[0, 1, 2, 4, 5, 6, 7, 8, 9, 10]
0.9622055397359565
1.6430364547128864
##########################################
[0, 1, 4, 5, 6, 7, 8, 9, 10, 14]
0.9614289412373803
1.4728011620439883
##########################################
[0, 1, 4, 5, 6, 7, 8, 9, 10, 12]
0.9679005953921822
1.467509401407011
##########################################
[0, 1, 4, 5, 6, 7, 8, 9, 10, 11]
0.9624644059021485
1.5381403295198603
##########################################
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
0.9658296660626456
1.330118382181314
##########################################
[0, 1, 4, 5, 6, 7, 8, 9, 10, 11, 12]
0.9679005953921822
1.4358173532642857
##########################################
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 12]
0.9717835878850635
1.2639811529178118
##########################################
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 12, 1

In [199]:
#rate,select_data = getCoverRate([0,1,4,5,6,7,8,9,10])

#rate,select_data = getCoverRate([0,1,2,3,4,5,6,7,8,9,10,11,12,13,14])
print(rate)

0.9720424540512556


In [ ]:
test_1 = np.array(pd.read_csv('testAll.csv'))
                  
a = []
for i in range(10):
    a.extend([train_pred_rf(select_data,train_loc_list)])
print(a)
print(np.array(a).mean())

In [74]:
getCoverRate([0,1,4,5,6,7,8,10])

0.9567693502459229

In [82]:
print('a')
print('b')

a
b


In [68]:
getCoverRate([0,1,2,3,4,5,6,7,8,9,10,11,12,13,14])

0.9720424540512556

In [59]:
select_data = getCoverRate([0,1,2,3])
len(select_data)
#len(select_data.columns)
col_len = len(select_data.columns)
row_len = len(select_data)
select_data = np.array(select_data)
a = []
for i in range(row_len):
    if select_data[i,:].max() <= -105:
        a.extend([i])
num = len(a)

0，
1
4
5
6
7
9
10
14


In [61]:
a


[185,
 196,
 197,
 198,
 209,
 250,
 251,
 252,
 253,
 254,
 255,
 256,
 257,
 258,
 259,
 260,
 261,
 262,
 263,
 287,
 288,
 289,
 305,
 342,
 343,
 344,
 345,
 346,
 347,
 348,
 349,
 350,
 351,
 352,
 353,
 354,
 355,
 356,
 357,
 358,
 359,
 360,
 361,
 362,
 363,
 364,
 365,
 366,
 367,
 368,
 369,
 370,
 371,
 372,
 373,
 374,
 375,
 376,
 377,
 378,
 379,
 380,
 381,
 382,
 436,
 437,
 438,
 439,
 440,
 441,
 442,
 443,
 444,
 445,
 446,
 447,
 448,
 449,
 450,
 451,
 452,
 453,
 454,
 455,
 456,
 457,
 458,
 459,
 460,
 461,
 462,
 463,
 464,
 465,
 466,
 467,
 468,
 469,
 470,
 471,
 472,
 473,
 474,
 475,
 488,
 528,
 529,
 530,
 531,
 532,
 533,
 534,
 535,
 536,
 537,
 538,
 539,
 540,
 541,
 542,
 543,
 544,
 545,
 546,
 547,
 548,
 549,
 550,
 551,
 552,
 553,
 554,
 555,
 556,
 557,
 558,
 559,
 560,
 561,
 562,
 563,
 564,
 565,
 566,
 567,
 568,
 619,
 620,
 621,
 622,
 623,
 624,
 625,
 626,
 627,
 628,
 629,
 630,
 631,
 632,
 633,
 634,
 635,
 636,
 637,
 638,
 639